In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
'''# TODO
# 
# something clever with deciding which/all sessions to look at 

# sessions = { # true for session(s) to graph.
#     "AFC"   : True,
#     "CRFT"  : False,
#     "EXT1"  : False,
#     "EXT2"  : False,
#     "RET"   : False,
#     "SR1"   : False,
#     "SR2"   : False,
#     "REN"   : False,
#     "RST"   : False
# }'''

In [ ]:
# TODO: stats
# see https://scipy-lectures.org/packages/statistics/index.html#student-s-t-test-the-simplest-statistical-test

In [2]:
session_data = "./data/ee1-2cfrt-mel.csv"

# TODO: CHANGE
ee1_vns = [2,3,6,7,9,11]
ee1_sham = [1,4,5,8,1]

ee2_vns
ee2_sham

vns_color = 'blue'
sham_color = 'red'

In [3]:
df = pd.read_csv(session_data)
# df

In [4]:
df.set_index("RAT",inplace=True)
df

,1,2,3,4
RAT,,,,
1,24,28,29,28
2,27,22,28,24
3,23,23,27,26
4,20,23,25,18
5,26,24,28,28
6,29,30,30,29
7,28,26,28,21
8,18,27,24,22
9,21,25,27,26


In [5]:
all_means = df.mean(axis=1)
all_means = 100*all_means/30

In [6]:
vns_means = means.loc[vns]
sham_means = means.loc[sham]

In [ ]:
# BAR PLOT OF MEANS, VNS VS SHAM

# Settings
w = 0.8  # bar width
x = (1,2.4)  # x coords of bars

groups = (vns_means, sham_means) # data to plot


grp = ("VNS","SHAM")  # labels for data

colors = (vns_color, sham_color)  # bar colors
size=(6,6)  # size of figure


# Graph
fig, ax = plt.subplots(figsize=size)

bars = (vns_means.mean(), sham_means.mean())  # bar height = group mean
error = (vns_means.std(), sham_means.std())  # error bar height = STD

ax.bar(
    x=x,
    height=bars,
    tick_label=grp,
    color=colors,
    width=w,
    yerr=error,
    capsize=12
)

ax.set(
    # xlim=(0,5),
    ylim=(0,100),
    yticks=range(0,120,20),
    ylabel="Freezing (%)"
)

for i in range(len(x)):
    ax.scatter(
        x=x[i]+np.zeros(len(groups[i])),
        y=groups[i],
        s=60,
        color="black",
        marker="o",
        facecolors="none",
        edgecolors="black"
        )

plt.show(fig)